First we need to load the blockchain file


In [2]:
import json
from typing import TypedDict, List

class BlockHeader(TypedDict):
    difficulty: int
    height: int
    miner: str
    nonce: int
    hash: str
    previous_block_header_hash: str
    timestamp: int
    transactions_count: int
    transactions_merkle_root: str
    
class Transaction(TypedDict):
    amount: int
    lock_time: int
    receiver: str
    sender: str
    signature: str
    transaction_fee: int 
    
class Block(TypedDict):
    header: BlockHeader
    transactions: List[Transaction]

    

In [3]:
blockchain_path = "data/blockchain.json"
with open(blockchain_path) as file:
   blocks : List[Block] = json.load(file)

We can test by printing the first header


In [4]:
    
print("----- Header -------")
print(blocks[0])
print("----- Transactions ---")
print(blocks[0]["transactions"])

----- Header -------
{'header': {'difficulty': 0, 'height': 0, 'miner': '0x0000000000000000000000000000000000000000', 'nonce': 0, 'hash': '0x9b714c0ae0b7d89a191c48e17e8f22db6926bb5abbce0bde658e8ee608d9861c', 'previous_block_header_hash': '0x0000000000000000000000000000000000000000000000000000000000000000', 'timestamp': 1697412600, 'transactions_count': 20, 'transactions_merkle_root': '0x0184e0b8556c2842399d3e6bcf8ff40f5d5d41a82f7e7e00e5f0a347f240dbdc'}, 'transactions': [{'amount': 22979000000, 'lock_time': 0, 'receiver': '0x1eb9f48d89a8c9313b6739cbe05f8c6aabae1c2a', 'sender': '0x0000000000000000000000000000000000000000', 'signature': '', 'transaction_fee': 0}, {'amount': 83138000000, 'lock_time': 0, 'receiver': '0xff82ae27911d16145c73992418a56f54abc068e0', 'sender': '0x0000000000000000000000000000000000000000', 'signature': '', 'transaction_fee': 0}, {'amount': 18008000000, 'lock_time': 0, 'receiver': '0xdd0a599ce6958eedc7e5cf83b83c01c7505b11dc', 'sender': '0x00000000000000000000000000

In [121]:
latest_block = blocks[-1]
latest_block_hash = latest_block["header"]["hash"]
print(f'Latest hash : {latest_block_hash}')
latest_block_time = latest_block["header"]["timestamp"]
print(f'Latest time : {latest_block_time}')
new_block_time = latest_block_time +10


Latest hash : 0x000a3698e344f2549cb88537e129a37ac78e195c08006486f928cc465bcc0550
Latest time : 1697413800


Now we need to extract all the transactions in the mempool that have a timelock less than the new block


In [48]:
with open("data/mempool.json") as file:
    mempool_transactions : List [Transaction] = json.load(file)

In [51]:
from collections import deque

In [123]:
from hashlib import sha256
def get_hash(val : str):
    return f'0x{sha256(val.encode()).hexdigest()}'

1a) Find the set of executable transactions

In [125]:

potential_transactions = []
for transaction in mempool_transactions: 
    if transaction["lock_time"] <= new_block_time:
        potential_transactions.append(transaction)
potential_transactions.sort(key= lambda x : x["transaction_fee"],reverse=True)
if len(potential_transactions)>100:
    potential_transactions = potential_transactions[:100]
potential_transactions= [{k: v for k, v in sorted(d.items())} for d in potential_transactions]

1b) Get the merkel root

In [ ]:
transaction_hashes = deque()
for potential_transaction in potential_transactions:
    transaction_hashes.append(get_hash((",".join(str(val) for val in potential_transaction.values()))))
null_hash = '0x' + 64*'0'
if len(transaction_hashes)==1:
    merkel_hash = get_hash(null_hash+transaction_hashes[0])
else:
    while len(transaction_hashes)!=1:
        current_length = len(transaction_hashes)
        is_odd = current_length %2 == 1
        for i in range(0,current_length,2):
            if is_odd and i == (current_length - 1):
                transaction_hashes.append(get_hash(null_hash + transaction_hashes.popleft()))
            else:
                a , b = transaction_hashes.popleft() , transaction_hashes.popleft()
                if b < a:
                    b , a = a , b
                transaction_hashes.append(get_hash(a+b))
    merkel_hash = transaction_hashes[0]
print(merkel_hash)

Here we create the potential new block , we still need to adjust the hash and nonce

In [145]:

new_block_header : BlockHeader = {
    "difficulty": min(((latest_block["header"]["height"] +1 )//50),6) ,
    "height": latest_block["header"]["height"]+1,
    "miner": "0x000a89667abeb2e87a42c724757ceee4cdc46eaa",
    "nonce": 0,
    "hash": "initial hash",
    "previous_block_header_hash": latest_block["header"]["hash"],
    "timestamp": latest_block["header"]["timestamp"]+10,
    "transactions_count": len(potential_transactions),
    "transactions_merkle_root": merkel_hash
}
new_block_transactions : List[Transaction] = potential_transactions

In [156]:
def hash_block_header(block_header : BlockHeader):
    temp_block_header =dict(sorted(block_header.items()))
    del temp_block_header["hash"]
    return get_hash(",".join(str(val) for val in temp_block_header.values()))
while hash_block_header(new_block_header)[2:2+new_block_header["difficulty"]]!="0"* new_block_header["difficulty"]:
    new_block_header["nonce"]+=1
new_block_header["hash"] = hash_block_header(new_block_header)
new_block : Block = {"header":new_block_header,"transactions":new_block_transactions}
blocks.append(new_block)
with open("data/new_blockchain.json", "w") as file:
    json.dump(blocks, file, indent=4)
